In [1]:
#4th goldmine at 7:30
import time
import threading
import pandas as pd
import os

# -----------------------------
# INITIAL GAME STATE
# -----------------------------
goldMine = 1                      # Start with 1 goldmine
initial_gold = 280.0
first_mine_cost = 150
gold = initial_gold - first_mine_cost  # Subtract cost of first Goldmine
multiplier = 5 + (goldMine / 2)
start_time = time.time()
running = True
lock = threading.Lock()

# -----------------------------
# DATA STORAGE
# -----------------------------
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes
auto_mine_3min_added = False
auto_mine_5min_added = False


# -----------------------------
# GOLD ACCUMULATION FUNCTION
# -----------------------------
def gold_function():
    global gold, multiplier, gold_history, running
    global auto_mine_3min_added, auto_mine_5min_added, goldMine
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 25 minutes have passed. Ending game.")
                running = False
                break

            # Auto-build at 3 minutes
            if not auto_mine_3min_added and elapsed_time >= 600:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_3min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 3 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 3 minutes failed: Not enough gold.")

            # Auto-build at 5 minutes
            if not auto_mine_5min_added and elapsed_time >= 1200:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_5min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 5 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 5 minutes failed: Not enough gold.")

            # Auto-build at 7.5 minutes (450 seconds)
            

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and accumulate gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Display output
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

# -----------------------------
# USER INPUT FUNCTION
# -----------------------------
def get_user_input():
    global gold, goldMine, running
    last_mine_time = start_time
    cooldown = 90

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# -----------------------------
# SAVE FUNCTION
# -----------------------------
def save_gold_history():
    global gold_history
    filename = "gold_progress.csv"
    run_id = 1

    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        gold_history.rename(columns={
            "gold": f"run_1_gold",
            "goldMine": f"run_1_goldMine"
        }, inplace=True)
        merged = gold_history

    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# -----------------------------
# MAIN THREAD HANDLING
# -----------------------------
try:
    function_thread = threading.Thread(target=gold_function)
    input_thread = threading.Thread(target=get_user_input)

    function_thread.start()
    input_thread.start()

    function_thread.join()
    input_thread.join()
except KeyboardInterrupt:
    with lock:
        running = False
        print("\n🛑 Keyboard interrupt detected. Exiting gracefully...")

# -----------------------------
# FINAL SAVE ON EXIT
# -----------------------------
with lock:
    save_gold_history()
    print("🎮 Game ended.")



--- Game Status ---
Time: 1 sec
Gold: 135
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 2 sec
Gold: 141
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 3 sec
Gold: 146
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 4 sec
Gold: 152
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 5 sec
Gold: 157
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 6 sec
Gold: 163
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 7 sec
Gold: 168
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 8 sec
Gold: 174
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 9 sec
Gold: 179
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 10 sec
Gold: 185
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 11 sec
Gold: 190
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 12 sec
Gold: 196
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 13 sec
Gold: 201
Multiplier: 5.5
Goldmines: 1

--- Game Status ---
Time: 14 sec
Gold: 207
Multiplier: 5.5



Enter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit:  quit


💾 Saving game data...
✅ Game data saved as Run #10 to gold_progress.csv
🎮 Game ended.
